<a href="https://colab.research.google.com/github/dany109/nayirep/blob/main/2020109_Lab_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%cu

#include <cuda_runtime.h>
#include <iostream>
#include <stdlib.h>
#include <time.h>

__global__ void matrixMulGlobal(float* A, float* B, float* C, int N) {
    int Row = blockIdx.y * blockDim.y + threadIdx.y;
    int Col = blockIdx.x * blockDim.x + threadIdx.x;

    if (Row < N && Col < N) {
        float Cvalue = 0;
        for (int k = 0; k < N; ++k) {
            Cvalue += A[Row*N + k] * B[k*N + Col];
        }
        C[Row*N + Col] = Cvalue;
    }
}
int main() {
    int N = 3; // Matrix size

    // host matrices A and B
    float h_A[N*N] = {1, 2, 3, 4, 5, 6, 7, 8, 9};
    float h_B[N*N] = {9, 15, 12, 52, -52, 15, -21, 52, 21};

    // device matrices A, B, and C
    float* d_A;
    cudaMalloc(&d_A, N*N*sizeof(float));
    cudaMemcpy(d_A, h_A, N*N*sizeof(float), cudaMemcpyHostToDevice);

    float* d_B;
    cudaMalloc(&d_B, N*N*sizeof(float));
    cudaMemcpy(d_B, h_B, N*N*sizeof(float), cudaMemcpyHostToDevice);

    float* d_C;
    cudaMalloc(&d_C, N*N*sizeof(float));

    // Launch the kernel
    dim3 dimGrid((N + 15) / 16, (N + 15) / 16, 1);
    dim3 dimBlock(16, 16, 1);
    clock_t t;
    t = clock();
    matrixMulGlobal<<<dimGrid, dimBlock>>>(d_A, d_B, d_C, N);
    cudaDeviceSynchronize();
    t = clock() - t;
    double time_taken = ((double)t)/CLOCKS_PER_SEC; // in seconds

    // resultdevice to the host
    float h_C[N*N];
    cudaMemcpy(h_C, d_C, N*N*sizeof(float), cudaMemcpyDeviceToHost);

    // Print the result
    for (int i = 0; i < N*N; ++i) {
        printf("%f ", h_C[i]);
        if (i % N == N-1) {
            printf("\n");
        }
    }
    printf("GPU with Global Memory took %f seconds to execute \n", time_taken);
    // Free memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}

0.000000 0.000000 0.000000 
0.000000 -0.000000 0.000000 
102011210997272476172787432554496.000000 -182365635117505337316245399432981381120.000000 224168.500000 
GPU with Global Memory took 0.000003 seconds to execute 



In [ ]:
pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-esygx8yx
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-esygx8yx
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4293 sha256=d3e151557cac009e5c622589d6b2871e3d07b6bd9d7e1929fbaf9b4d64baab1c
  Stored in directory: /tmp/pip-ephem-wheel-cache-bdu_ir0g/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out
